In [1]:
##Download data
# !wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet
# !wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet

--2022-05-17 08:51:24--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 54.231.132.113
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|54.231.132.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11886281 (11M) [binary/octet-stream]
Saving to: ‘fhv_tripdata_2021-01.parquet’

fhv_tripdata_2021-0 100%[===================>]  11.33M  27.8MB/s    in 0.4s    

2022-05-17 08:51:25 (27.8 MB/s) - ‘fhv_tripdata_2021-01.parquet’ saved [11886281/11886281]

--2022-05-17 08:51:25--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 54.231.132.113
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|54.231.132.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10645466 (10M) [binary/octet-stream]
Saving to: ‘fhv_tripdata_2021-02.parquet’

fhv

In [76]:
##Install dependencies
# !pip install pyarrow
# !pip install sklearn

In [52]:
##Import packages
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error

In [53]:
jan = pd.read_parquet('fhv_tripdata_2021-01.parquet')

Question 1:

In [4]:
jan.shape[0]

1154112

In [5]:
jan.columns

Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number'],
      dtype='object')

In [54]:
#Create duration column
jan['duration'] = jan.dropOff_datetime - jan.pickup_datetime

In [55]:
#Convert duration column to minutes
jan.duration = jan.duration.apply(lambda td: td.total_seconds()/60)

Question 2:

In [8]:
jan.duration.mean()

19.167224093791006

In [56]:
#Limit the data set to trips greater or equal to 1 minute and less than or equal to 60 minutes
jan_limited = jan[(jan.duration >= 1) & (jan.duration <= 60)]

Question 2.5:

In [10]:
jan.shape[0]-jan_limited.shape[0]

44286

In [57]:
#Fill missing location ID values with -1
jan_limited['PUlocationID']=jan_limited['PUlocationID'].fillna(-1)
jan_limited['DOlocationID']=jan_limited['DOlocationID'].fillna(-1)

<ipython-input-57-1a2c50f447d9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_limited['PUlocationID']=jan_limited['PUlocationID'].fillna(-1)
<ipython-input-57-1a2c50f447d9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_limited['DOlocationID']=jan_limited['DOlocationID'].fillna(-1)


In [58]:
len(jan_limited.loc[jan_limited['DOlocationID'] == -1])

147907

Question 3:

In [59]:
#Get percentage of missing PUlocationID values
len(jan_limited.loc[jan_limited['PUlocationID'] == -1])/len(jan_limited['PUlocationID'])

0.8352732770722617

In [15]:
jan_limited.shape

(1109826, 8)

In [60]:
categorical = ['PUlocationID','DOlocationID']

In [61]:
jan_limited.dtypes

dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                     float64
DOlocationID                     float64
SR_Flag                           object
Affiliated_base_number            object
duration                         float64
dtype: object

In [62]:
jan_limited['PUlocationID']=jan_limited['PUlocationID'].astype(str)
jan_limited['DOlocationID']=jan_limited['DOlocationID'].astype(str)

<ipython-input-62-ea0f7957599f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_limited['PUlocationID']=jan_limited['PUlocationID'].astype(str)
<ipython-input-62-ea0f7957599f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_limited['DOlocationID']=jan_limited['DOlocationID'].astype(str)


In [63]:
#Create a dict vectorizer
dv = DictVectorizer()

In [64]:
#Make a dictionary from the categorical columns defined to feed to the DictVectorizer
train_dict = jan_limited[categorical].to_dict(orient='records')

In [66]:
#Create the training set from the DictVectorizer
X_train = dv.fit_transform(train_dict)

Question 4:

In [67]:
X_train.shape[1]

525

In [68]:
#Define target column you are predicting
target = 'duration'
y_train = jan_limited[target].values

In [69]:
from sklearn.linear_model import LinearRegression

In [70]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [71]:
y_pred = lr.predict(X_train)

Question 5:

In [72]:
mean_squared_error(y_train,y_pred,squared=False)

10.5285191072053

In [73]:
#Read in validation set
feb = pd.read_parquet('fhv_tripdata_2021-02.parquet')

In [74]:
#Create duration column
feb['duration'] = feb.dropOff_datetime - feb.pickup_datetime

In [75]:
#Convert the duration column to minutes
feb.duration = feb.duration.apply(lambda td: td.total_seconds()/60)

In [76]:
#Limit the data set to trips greater or equal to 1 minute and less than or equal to 60 minutes
feb_limited = feb[(feb.duration >= 1) & (feb.duration <= 60)]

In [77]:
#Fill missing location ID values with -1
feb_limited['PUlocationID']=feb_limited['PUlocationID'].fillna(-1)
feb_limited['DOlocationID']=feb_limited['DOlocationID'].fillna(-1)

<ipython-input-77-401082b982aa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feb_limited['PUlocationID']=feb_limited['PUlocationID'].fillna(-1)
<ipython-input-77-401082b982aa>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feb_limited['DOlocationID']=feb_limited['DOlocationID'].fillna(-1)


In [78]:
feb_limited['PUlocationID']=feb_limited['PUlocationID'].astype(str)
feb_limited['DOlocationID']=feb_limited['DOlocationID'].astype(str)

<ipython-input-78-726b2329b9cb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feb_limited['PUlocationID']=feb_limited['PUlocationID'].astype(str)
<ipython-input-78-726b2329b9cb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feb_limited['DOlocationID']=feb_limited['DOlocationID'].astype(str)


In [79]:
feb_train_dict = feb_limited[categorical].to_dict(orient='records')

In [80]:
X_Test = dv.transform(feb_train_dict)
y_test = feb_limited[target].values

In [81]:
y_pred = lr.predict(X_Test)

Question 6:

In [82]:
mean_squared_error(y_test,y_pred,squared=False)

11.014283143790387

11.014283143790387

In [166]:
def prep_file(file):
    df = pd.read_parquet(file)
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)
    df['PUlocationID']=df['PUlocationID'].fillna(-1)
    df['DOlocationID']=df['DOlocationID'].fillna(-1)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    return df
    

In [165]:
def make_predictions(df):
    categorical = ['PUlocationID','DOlocationID']
    target = 'duration'
    x_dict = df[categorical].to_dict(orient='records')
    X_Train = dv.fit_transform(x_dict)
    y_train = df[target].values
    lr = LinearRegression()
    lr.fit(X_train,y_train)
    return lr.predict(X_train)

In [167]:
y_preds = make_predictions(prep_file('fhv_tripdata_2021-01.parquet'))

In [ ]:
y_train = prep_file('fhv_tripdata_2021-01.parquet').duration.values

In [168]:
mean_squared_error(y_train,y_preds,squared=False)

11.415432830521663